In [110]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
import re, string
from nltk.corpus import stopwords
from nltk.corpus import twitter_samples
from nltk import classify
from nltk import NaiveBayesClassifier
import random
from sklearn.metrics import classification_report

In [54]:
positive_tweets  = twitter_samples.strings('positive_tweets.json')
negative_tweets  = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')

In [109]:
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
# tweet_tokens[2]

In [56]:

# print(pos_tag(tweet_tokens[1]))

def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos ='n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

print(lemmatize_sentence(tweet_tokens[2]))

stop_words=stopwords.words('english')
def remove_noise(tweet_tokens, stop_words=()):
    cleaned_tweets =[]
    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        
        if tag.startswith('NN'):
            pos='n'
        elif tag.startswith('VB'):
            pos='v'
        else:
            pos='a'
        lemmatizer=WordNetLemmatizer()
        token = lemmatizer.lemmatize(token,pos)
        
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tweets.append(token.lower())
    return cleaned_tweets
    
    
print(remove_noise(tweet_tokens[2], stop_words))

positive_uncleaned_tweets=twitter_samples.tokenized('positive_tweets.json')
positive_cleaned_tweets_list=[]
negative_uncleaned_tweets=twitter_samples.tokenized('negative_tweets.json')
negative_cleaned_tweets_list=[]

for words in positive_uncleaned_tweets:
    positive_cleaned_tweets_list.append(remove_noise(words, stop_words ))
    
for words in negative_uncleaned_tweets:
    negative_cleaned_tweets_list.append(remove_noise(words, stop_words ))
    
def model_for_tweets(cleaned_tokens_list):
    for tweets in cleaned_tokens_list:
        yield dict([tweet, True] for tweet in tweets )
        
positive_tweets_model=model_for_tweets(positive_uncleaned_tweets)
negative_tweets_model=model_for_tweets(negative_uncleaned_tweets)

positive_dataset=[(tweet_dict, 'Positive') for tweet_dict in positive_tweets_model]
negative_dataset=[(tweet_dict, 'Negative') for tweet_dict in negative_tweets_model]

dataset=positive_dataset+negative_dataset
random.shuffle(dataset)

train_data=dataset[:7000]
test_data=dataset[7000:]

['@DespiteOfficial', 'we', 'have', 'a', 'listen', 'last', 'night', ':)', 'As', 'You', 'Bleed', 'be', 'an', 'amazing', 'track', '.', 'When', 'be', 'you', 'in', 'Scotland', '?', '!']
['listen', 'last', 'night', ':)', 'bleed', 'amazing', 'track', 'scotland']


In [57]:
# print(positive_uncleaned_tweets[13])
# print(positive_cleaned_tweets_list[13])

In [111]:

classifier = NaiveBayesClassifier.train(train_data)
print("Accuracy is : ", classify.accuracy(classifier, test_data))
print(classifier.show_most_informative_features(10))
report  = classification_report(classifier, test_data['Label'], output_dict=True)
print(report['Positive'])

Accuracy is :  0.9943333333333333
Most Informative Features
                      :( = True           Negati : Positi =   2074.2 : 1.0
                      :) = True           Positi : Negati =    989.1 : 1.0
                     sad = True           Negati : Positi =     46.7 : 1.0
                  THANKS = True           Negati : Positi =     27.2 : 1.0
                   Thank = True           Positi : Negati =     25.1 : 1.0
                     See = True           Positi : Negati =     24.1 : 1.0
                    THAT = True           Negati : Positi =     23.2 : 1.0
                   loves = True           Positi : Negati =     22.8 : 1.0
                  FOLLOW = True           Negati : Positi =     21.8 : 1.0
                  Thanks = True           Positi : Negati =     21.4 : 1.0
None


TypeError: list indices must be integers or slices, not str

### Testing with custom tweet

In [83]:
custom_tweet="""
During opening statements in Derek Chauvin's trial in the death of George Floyd, prosecutors showed jurors a bystander video of the former officer kneeling on Floyd's neck.
"""
custom_tokens = remove_noise(nltk.word_tokenize(custom_tweet))
x = classifier.classify(dict([token, True] for token in custom_tokens))
x

neg
Negative


In [107]:
from tkinter import *

window = Tk()
window.title("Sentiment Analysis App")
window.geometry('500x400')

def analyze():
    msg =remove_noise(nltk.word_tokenize( tweetTxtArea.get("1.0",'end-1c')))
#     msg_token=  remove_noise(nltk.word_tokenize(msg))
    sentiment_result = (classifier.classify(dict([token, True] for token in msg)))
    if sentiment_result == 'Positive':
        sentiment_label.configure(text=sentiment_result,  fg='green')
    else:
         sentiment_label.configure(text=sentiment_result, fg='red')

label = Label(window, text='Tweet Sentiment Analysis',font=(24)) # label for heading
label.grid(row=0, column=1)
label = Label(window, text='Enter tweet',) #label for enter tweet
label.grid(row=1, column=1)
tweetTxtArea = Text(window, height=10, width=60 ) # textarea
tweetTxtArea.grid(row=2, column = 1, padx=10)
sentiment_label = Label(window, text='Tweet is ...') # sentiment label
sentiment_label.grid(row=3, column=1, pady=5)
btn_analyze = Button(window, text='Analyze Tweet', bg='deepskyblue' ,command=analyze) # button for analysis
btn_analyze.grid(row=4, column=1)


window.mainloop()